## Inference by Gibbs sampling

## *Following section is in progress, not finished/correct yet*

### Finite model

The tutorial states that we need $P(z_{ik} = 1 \mid \mathbf{z}_{-i,k})$, and we can get this by integrating equation (10) over $\pi_k$.  Equation (10) is:

$$
P(\mathbf{Z}) = \prod_{k=1}^K
\frac
  {\frac{\alpha}{K} \Gamma(m_k + \frac{\alpha}{K})\Gamma(N - m_k + 1)}
  {\Gamma(N + 1 + \frac{\alpha}{K})}
$$

The tutorial states that we should obtain $P(z_{ik} = 1\mid \mathbf{z}_{-i,k})$ using the following integral:

$$
\int_0^1 P(z_{ik} \mid \pi_k)\, p(\pi_k \mid \mathbf{z}_{-i,k})\, d\pi_k
$$

So, we are finding the probability of one specific feature, for one specific customer, given the feature assignments to all other customer/feature pairs, and integrating over all values of $\pi_k$.  Note that $\pi_k$ is only conditional on $\mathbf{z}_{-i,k}$, dont need to consider $\mathbf{z}$ for other features, since they dont give any information on the latent variable $\pi_k$. Each $\pi_k$ is conditionally independent of other $\pi_k$, given the prior distribution, ie we are using Beta, and its hyperparameters $(\alpha/K, 1)$.

I'm not sure I quite see how to do this though.  It seems like we need to use Bayes Rule to reverse the prior of $P(\mathbf{z}_{-i,k} \mid \pi_k)$ first?  Bayes rule here would be:

$$
p(\pi_k \mid \mathbf{z}_{-i,k}) =
\frac
  {P(\mathbf{z}_{-i,k} \mid \pi_k)\,p(\pi_k)}
  {P(\mathbf{z}_{-i,k})}
$$

Substituting into the integral above, we get:

$$
\int_0^1 P(z_{ik} = 1 \mid \pi_k)\, \frac
  {P(\mathbf{z}_{-i,k} \mid \pi_k)\,p(\pi_k)}
  {P(\mathbf{z}_{-i,k})}
\, d\pi_k
$$

$$
=
\int_0^1
  \pi_k\, \frac
  {P(\mathbf{z}_{-i,k} \mid \pi_k)\,p(\pi_k)}
  {P(\mathbf{z}_{-i,k})}
\, d\pi_k
$$

$P(\mathbf{z}_{-i,k})$ is independent of $z_{ik}$, is just a normalizing constant, wont affect the shape of the distribution of $z_{ik}$, so we can replace it with proportionality:

$$
\propto
\int_0^1
  \pi_k\,
  P(\mathbf{z}_{-i,k} \mid \pi_k)\,p(\pi_k)
\, d\pi_k
$$

$P(\mathbf{z}_{-i,k})$ is:

$$
\prod_{j=1}^N \pi_k^{z_{j,k}(1-\delta(i,j)}
\cdot
\prod_{j=1}^N (1 - \pi_k)^{(1 - z_{j,k})(1-\delta(i,j)}
$$

(where $\delta(i,j)$ is the Kronecker delta)

Using $m_{-i,k}$ to denote:

$$
m_{-i,k} = \sum_{j=1}^N (1 - \delta(i,j))z_{j,k}
$$

... then $P(\mathbf{z}_{-i,k})$ is:

$$
\prod_{j=1}^N \pi_k^{z_{j,k}(1-\delta(i,j))}
\cdot
\prod_{j=1}^N (1 - \pi_k)^{(1 - z_{j,k})(1-\delta(i,j)}
$$

$$
=
\pi_k^{\sum_{j=1}^N z_{j,k}(1-\delta(i,j))}
\cdot
(1 - \pi_k)^{
   \sum_{j=1}^N (1-z_{j,k})(1-\delta(i,j)
}
$$

$$
=
\pi_k^{\sum_{j=1}^N z_{j,k}(1-\delta(i,j))}
\cdot
(1 - \pi_k)^{
   N - 1 - \sum_{j=1}^N (z_{j,k})(1-\delta(i,j)
}
$$

$$
=
\pi_k^{m_{-i,k}}
\cdot
(1 - \pi_k)^{
   N - 1 - m_{-i,k}
}
$$


$p(\pi_k)$ is the prior on $\pi_k$, which is $\mathrm{Beta}(pi_k; \alpha/K, 1)$:

$$
p(\pi_k)
=
\mathrm{Beta}(\pi_k; \alpha/K, 1)
$$
$$
=
\frac{\pi_k^{\alpha/K - 1} (1-\pi_k)^{1 - 1}}
  {B(\alpha/K, 1)}
$$
$$
=
\frac{\pi_k^{\alpha/K - 1}}
   {B(\alpha/K, 1)}
$$
$$
=
\frac{\alpha}{K}
\pi_k^{\alpha/K - 1}
$$


So, the integral becomes:

$$
\int_0^1
\pi_k
\cdot
\pi_k^{\alpha/K - 1}
\cdot
\pi_k^{m_{-i,k}}
\cdot
(1 - \pi_k)^{
   N - 1 - m_{-i,k}
}
\, d\pi_k
$$

$$
=
\int_0^1
\pi_k^{\alpha/K + m_{-i,k}}
\cdot
(1 - \pi_k)^{
   N - 1 - m_{-i,k}
}
\, d\pi_k
$$

which integrates into the Beta function, following the pattern:

$$
B(r, s)
= \int_0^1 \pi_k^{r-1} (1-\pi_k)^{s-1}\, d\pi_k
$$



Giving:

$$
B(\alpha/K + m_{-i,k} + 1, N - m_{-i,k})
$$


$$
=
\frac{\Gamma(\alpha/K + m_{-i,k} + 1) \Gamma(N - m_{-i,k})}
  {\Gamma(\alpha/K + N + 1)}
$$